In [269]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [270]:
sms = pd.read_csv('spam.csv', encoding='latin-1')
sms.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [271]:
# X = df[['FirstDigitTouch','BothStartLoadPhase']].values
# y = df['LiftOff'] == "a"
# pipe = Pipeline([
#     ("scale",StandardScaler()),
#     ("model",LinearRegression())
# ])
# pipe.get_params
sms=sms.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1)
sms=sms.rename(columns={'v1':'labels','v2': 'message'})
sms.head()

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [272]:
# parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [273]:
# plt.scatter(X[:, 0],X[:,1], c=y);
# mod = GridSearchCV(estimator=pipe,cv=3,)
#                   param_grid={'': [1,2,3,4,5,6,7,8,9,10]},
       #           cv=3)

In [274]:
# sms['labels']=df.labels.map({'spam':0, 'ham':1})
# sms.head()

In [275]:
sms['length']=sms['message'].apply(len)
sms.head()

,labels,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [276]:
# plt.figure(figsize=(16,6))
# sns.distplot(a=df['length'])
# # plt.legend()

In [277]:
# asd=df[df['length']==111]['sms'].iloc[0]
# asd
text_feat = sms['message'].copy()

In [278]:
# text_feat = df['message'].copy()
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

In [279]:
text_feat = text_feat.apply(text_process)

In [280]:
vectorizer = TfidfVectorizer()

In [281]:
features = vectorizer.fit_transform(text_feat)


In [282]:
features_train, features_test, labels_train, labels_test = train_test_split(features, sms['labels'], test_size=0.3, random_state=111)

In [283]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
lrc = LogisticRegression(solver='liblinear', penalty='l1')

In [284]:
clfs = {'SVC' : svc,'KN' : knc,'LR': lrc}


In [285]:
def train_classifier(clf, feature_train, labels_train):    
    clf.fit(feature_train, labels_train)

In [286]:
def predict_labels(clf, features):
    return (clf.predict(features))

In [291]:
pred_scores = []
for k,v in clfs.items():
    train_classifier(v, features_train, labels_train)
    pred = predict_labels(v,features_test)
    pred_scores.append((k, [accuracy_score(labels_test,pred)]))
pred_scores

[('SVC', [0.9784688995215312]),
 ('KN', [0.9246411483253588]),
 ('LR', [0.9431818181818182])]

In [292]:
df = pd.DataFrame(pred_scores)
df 

,0,1
0,SVC,[0.9784688995215312]
1,KN,[0.9246411483253588]
2,LR,[0.9431818181818182]
